<a href="https://colab.research.google.com/github/yugamjayant/sept_2023_jh/blob/main/Movie_recommendation_sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DL based movie recommendation system

Movie lens Dataset(small) :https://grouplens.org/datasets/movielens/latest/

We will build a reccomender system using content filtering algorithm

In [ ]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
import re
import copy

In [ ]:
links_rawds = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/ml-latest-small/links.csv")
movies_rawds = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ml-latest-small/movies.csv')
ratings_rawds = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ml-latest-small/ratings.csv')
tags_rawds = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ml-latest-small/tags.csv')

Data Preprocessing

In [ ]:
#Lets prepare data that can be fed to the model
#Lets start by preaparing User side of data

movie_genres = copy.deepcopy(movies_rawds)

all_genres = set()
for index, row in movies_rawds.iterrows():
  all_genres.update(row['genres'].split('|'))

#Remove "(no genres listed)" from genres list
all_genres.discard('(no genres listed)')


#Adding new columns, denoting presence of each genre for each movie

for genre in all_genres:
  movie_genres[genre] = movie_genres['genres'].str.contains(genre)


#Merging the dataframes, rating rewards dataframe and movie genres dataframe

ratings_movie_genres = pd.merge(left = ratings_rawds[['userId', 'movieId', 'rating']],
                                right = movie_genres,
                                on = 'movieId',
                                how = 'left')


#Function that renames columns, it helps in assigning new column names after aggregatte function
def create_col_names(agg_dict, agg_col):
  col_names = [agg_col]
  for key in agg_dict.keys():
    value = agg_dict[key]
    if type(value) is list:
      for value_1 in value:
        col_names.append(key+'_'+value_1)
    elif type(value) != str:
      col_names.append(key+'_'+'cfunc')
    else:
      col_names.append(key+'_'+value)

  return col_names


#This function calculates genre wise avr ratings
def genre_wise_avg_rating(array_temp):

  sum_ratings = array_temp.sum()
  #Non zero ratings count
  array_temp_summary = array_temp.value_counts()
  non_zero_ratings_count = array_temp_summary.sum()

  if 0 in array_temp_summary.index:
    non_zero_ratings_count = array_temp_summary.sum() - array_temp_summary[0]

  return sum_ratings/non_zero_ratings_count



#Adding new columns, denoting presence of each genre for each movie

for genre in all_genres:
  movie_genres[genre] = movie_genres['genres'].str.contains(genre)



#Merging the dataframes, rating rewards dataframe and movie genres dataframe

ratings_movie_genres = pd.merge(left = ratings_rawds[['userId', 'movieId', 'rating']],
                                right = movie_genres,
                                on = 'movieId',
                                how = 'left')


#Function that renames columns, it helps in assigning new column names after aggregatte function
def create_col_names(agg_dict, agg_col):
  col_names = [agg_col]
  for key in agg_dict.keys():
    value = agg_dict[key]
    if type(value) is list:
      for value_1 in value:
        col_names.append(key+'_'+value_1)
    elif type(value) != str:
      col_names.append(key+'_'+'cfunc')
    else:
      col_names.append(key+'_'+value)

  return col_names


global user_genre_ratings_array

#This function calculates genre wise avr ratings
def genre_wise_avg_rating(array_temp):
  global user_genre_ratings_array
  user_genre_ratings_array = array_temp
  sum_ratings = array_temp.sum()
  #Non zero ratings count
  array_temp_summary = array_temp.value_counts()
  non_zero_ratings_count = array_temp_summary.sum()

  if 0 in array_temp_summary.index:
    non_zero_ratings_count = array_temp_summary.sum() - array_temp_summary[0]

  return sum_ratings/non_zero_ratings_count

#Adding rating of each movie to genre column, to facilitate averages in thenext step
for genre in all_genres:
  ratings_movie_genres[genre] = ratings_movie_genres[genre].astype(float) * ratings_movie_genres['rating']


#Aggregating the dataframe to compute average ratings for each genre
movie_genre_df_agg_dict = {'rating':['mean', 'sum'], 'movieId':'count', 'War': genre_wise_avg_rating, 'Adventure': genre_wise_avg_rating, 'Romance': genre_wise_avg_rating, 'Musical': genre_wise_avg_rating, 'Western': genre_wise_avg_rating, 'Fantasy': genre_wise_avg_rating, 'Children': genre_wise_avg_rating, 'Sci-Fi': genre_wise_avg_rating, 'Drama': genre_wise_avg_rating, 'Crime': genre_wise_avg_rating, 'Mystery': genre_wise_avg_rating, 'Horror': genre_wise_avg_rating, 'Documentary': genre_wise_avg_rating, 'Action': genre_wise_avg_rating, 'Comedy': genre_wise_avg_rating, 'Thriller': genre_wise_avg_rating, 'Animation': genre_wise_avg_rating, 'Film-Noir': genre_wise_avg_rating, 'IMAX': genre_wise_avg_rating}

ratings_movie_genres_int = ratings_movie_genres.groupby('userId').aggregate(movie_genre_df_agg_dict).reset_index()

ratings_movie_genres_int.columns = create_col_names(movie_genre_df_agg_dict, 'userId')

ratings_movie_genres_int.fillna(0, inplace = True)


#Expanding the df such that we our number of rows matches the number of ratings
ratings_movie_genres_int2 = pd.merge(ratings_rawds[['userId']], ratings_movie_genres_int, on = 'userId', how = 'left')


#Excluded few unnecessary columns
user_ratings_df = ratings_movie_genres_int2[['userId','movieId_count', 'rating_mean', 'Action_cfunc', 'Western_cfunc', 'Sci-Fi_cfunc', 'IMAX_cfunc', 'Drama_cfunc',
       'Thriller_cfunc', 'Adventure_cfunc', 'Animation_cfunc', 'Musical_cfunc', 'Mystery_cfunc', 'Fantasy_cfunc',
       'Children_cfunc', 'Film-Noir_cfunc', 'Romance_cfunc', 'Crime_cfunc', 'War_cfunc', 'Documentary_cfunc',
       'Horror_cfunc', 'Comedy_cfunc']]


#Now Preparing item side of dataframe







#We needed year in the dataframe, thus the following function extracts year from movie title

def extract_year(title):
  try:
    year = re.search("\(([0-9]{4})\)", title).group().strip('()')

  except Exception as e:
    print("Error: ", e)
    year = 0

  return int(year)


movie_genres['year'] = movie_genres['title'].apply(lambda x: extract_year(x))


#Expanding the df such that we our number of rows matches the number of ratings

movie_genres_ratings = pd.merge(ratings_rawds[['movieId']], movie_genres, on = 'movieId', how = 'left')

for genre in all_genres:
  movie_genres_ratings[genre] = movie_genres_ratings[genre].astype(int)


movie_genres_ratings.drop(columns = ['genres'], inplace = True)

#calculating avg ratings for each movie
avg_ratings_movie_wise = ratings_rawds.groupby('movieId').aggregate({'rating':'mean'}).reset_index().rename(columns = {'rating':'mean_ratings'})

#Adding avg rating(per movie) column to movie_genres_ratings
movie_genres_ratings = pd.merge(movie_genres_ratings, avg_ratings_movie_wise[['movieId','mean_ratings']], on = 'movieId', how = 'left')

item_ratings_df = movie_genres_ratings[['movieId', 'year','mean_ratings','Action', 'Western', 'Sci-Fi', 'IMAX', 'Drama',
       'Thriller', 'Adventure', 'Animation', 'Musical', 'Mystery', 'Fantasy',
       'Children', 'Film-Noir', 'Romance', 'Crime', 'War', 'Documentary',
       'Horror', 'Comedy']]



In [ ]:
#Creating user and item numpy arrays, here we have sliced the dataframe to remove the index column

item_train = item_ratings_df.to_numpy()[:,1:]
user_train = user_ratings_df.to_numpy()[:,1:]
y_train = ratings_rawds['rating'].to_numpy()

In [ ]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

#np.allclose, Returns True if two arrays are element-wise equal within a tolerance.

True
True


In [ ]:
#Test train split

item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (80668, 21)
movie/item test data shape: (20168, 21)


Defining the model Neural Network architecture for content-based filtering

In [ ]:

num_outputs = 32
num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
      tf.keras.layers.Dense(256, activation = 'relu'),
      tf.keras.layers.Dense(128, activation = 'relu'),
      tf.keras.layers.Dense(num_outputs, activation = 'linear')

    ### END CODE HERE ###
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
      tf.keras.layers.Dense(256, activation = 'relu'),
      tf.keras.layers.Dense(128, activation = 'relu'),
      tf.keras.layers.Dense(num_outputs, activation = 'linear')


    ### END CODE HERE ###
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 18)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 20)]                 0         []                            
                                                                                                  
 sequential_2 (Sequential)   (None, 32)                   41888     ['input_1[0][0]']             
                                                                                                  
 sequential_3 (Sequential)   (None, 32)                   42400     ['input_2[0][0]']             
                                                                                              

In [ ]:
#We will use a mean squared error loss and an Adam optimizer.

tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [ ]:
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items

tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
2521/2521 [==============================] - 13s 4ms/step - loss: 0.1356
Epoch 2/30
2521/2521 [==============================] - 8s 3ms/step - loss: 0.1294
Epoch 3/30
2521/2521 [==============================] - 9s 4ms/step - loss: 0.1267
Epoch 4/30
2521/2521 [==============================] - 9s 3ms/step - loss: 0.1251
Epoch 5/30
2521/2521 [==============================] - 8s 3ms/step - loss: 0.1237
Epoch 6/30
2521/2521 [==============================] - 10s 4ms/step - loss: 0.1224
Epoch 7/30
2521/2521 [==============================] - 8s 3ms/step - loss: 0.1208
Epoch 8/30
2521/2521 [==============================] - 9s 4ms/step - loss: 0.1194
Epoch 9/30
2521/2521 [==============================] - 9s 4ms/step - loss: 0.1186
Epoch 10/30
2521/2521 [==============================] - 8s 3ms/step - loss: 0.1179
Epoch 11/30
2521/2521 [==============================] - 9s 4ms/step - loss: 0.1173
Epoch 12/30
2521/2521 [==============================] - 8s 3ms/step - loss: 0.1169

In [ ]:
#Evaluate the model to determine loss on the test data.

model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

631/631 [==============================] - 2s 3ms/step - loss: 0.1179


0.11787594109773636

Here we'll take user parameters for our target users to churn out predictions

In [ ]:
#Predictions for a new user
#First, we'll create a new user and have the model suggest movies for that user.

new_user_id = 5000
new_rating_ave = 0.0
new_Action = 1
new_Western = 2
new_Sci_Fi = 4
new_IMAX = 0
new_Drama = 2
new_Thriller = 2
new_Adventure = 2
new_Animation = 0
new_Musical = 0
new_Mystery = 1
new_Fantasy = 1
new_Children = 0
new_Film_Noir = 0
new_Romance = 0
new_Crime = 1
new_War = 0
new_Documentary = 4
new_Horror = 0
new_Comedy = 1
new_rating_count = 3


# 'Action', 'Western', 'Sci-Fi', 'IMAX', 'Drama',
#        'Thriller', 'Adventure', 'Animation', 'Musical', 'Mystery', 'Fantasy',
#        'Children', 'Film-Noir', 'Romance', 'Crime', 'War', 'Documentary',
#        'Horror', 'Comedy'

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_Action, new_Western, new_Sci_Fi, new_IMAX, new_Drama,
                      new_Thriller, new_Adventure, new_Animation, new_Musical, new_Mystery, new_Fantasy,
                      new_Children, new_Film_Noir, new_Romance, new_Crime, new_War, new_Documentary,
                      new_Horror, new_Comedy]])

In [ ]:
def print_pred_movies(y_p, item, movie_dict, maxcount=10):
    """ print results of prediction of a new user. inputs are expected to be in
        sorted order, unscaled. """
    count = 0
    disp = [["y_p", "movie id", "rating ave", "title", "genres"]]

    for i in range(0, y_p.shape[0]):
        if count == maxcount:
            break
        count += 1
        movie_id = item[i, 0].astype(int).astype(str)
        disp.append([np.around(y_p[i, 0], 1), item[i, 0].astype(int), np.around(item[i, 2].astype(float), 1),
                     movie_dict[movie_id]['title'], movie_dict[movie_id]['genres']])

    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
    return table

Movie reccomendations for our new user

In [ ]:
#9742 is number of movies
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = np.tile(user_vec[:,1:], (9742, 1))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

display(HTML(print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)))

305/305 [==============================] - 1s 2ms/step


y_p,movie id,rating ave,title,genres
3.8,2014,1,Freaky Friday (1977),Children|Comedy|Fantasy
3.8,1980,1,Friday the 13th Part VII: The New Blood (1988),Horror
3.7,1972,1,"Nightmare on Elm Street 5: The Dream Child, A (1989)",Horror
3.7,1975,1,Friday the 13th Part 2 (1981),Horror
3.7,1951,0,Oliver! (1968),Drama|Musical
3.7,1957,0,Chariots of Fire (1981),Drama
3.7,2016,1,"Apple Dumpling Gang Rides Again, The (1979)",Children|Comedy|Western
3.7,2017,1,Babes in Toyland (1961),Children|Fantasy|Musical
3.7,2007,1,Polish Wedding (1998),Comedy
3.7,1997,1,"Exorcist, The (1973)",Horror|Mystery
